In [1]:
import pandas as pd
import torch

from netam import framework, models
from netam.common import nt_mask_tensor_of, BASES

import sys
sys.path.append("/Users/matsen/re/netam-experiments-1")
from shmex.shm_data import pcp_df_of_non_shmoof_nickname

In [2]:
def mutation_count_of_row(row):
    return sum(
        1 for p, c in zip(row["parent"], row["child"]) if p != c and p != "N" and c != "N"
    )    

def sim_df_of_pcp_and_crepe(pcp_nickname, crepe_path, sim_row_count=10000):
    pcp_df = pcp_df_of_non_shmoof_nickname(pcp_nickname, sample_count=sim_row_count)

    pcp_df['mutation_count'] = pcp_df.apply(mutation_count_of_row, axis=1)
    sim_df = pcp_df[["parent", "branch_length", "mutation_count"]]
    sim_df = sim_df[sim_df["mutation_count"] > 0].reset_index(drop=True)
    sim_df["parent_mask"] = sim_df["parent"].apply(lambda x: nt_mask_tensor_of(x, 500))

    crepe = framework.load_crepe(crepe_path)
    ratess, csp_logitss = crepe(sim_df["parent"])

    children = []

    for i, row in sim_df.iterrows():
        parent = row["parent"]
        mutation_count = row["mutation_count"]
        parent_mask = row["parent_mask"]
        rates = ratess[i]
        csp_logits = csp_logitss[i]
        rates *= parent_mask
        mut_idxs = torch.multinomial(rates, mutation_count, replacement=False)
        child = list(parent)
        for i in mut_idxs:
            child[i] = BASES[torch.multinomial(torch.exp(csp_logits[i]), 1)]
        children.append("".join(child))

    sim_df["child"] = children
    sim_df["simulated_mut_count"] = sim_df.apply(mutation_count_of_row, axis=1)
    assert (sim_df["simulated_mut_count"] == sim_df["mutation_count"]).all()
    sim_df = sim_df.drop(columns=["mutation_count", "parent_mask", "simulated_mut_count"])
    return sim_df

In [3]:
data_nickname = "shmoof"
crepe_path = "../train/trained_models/cnn_joi_lrg-shmoof_small-fixed-0"
sim_df = sim_df_of_pcp_and_crepe(data_nickname, crepe_path)
sim_df.to_csv("~/data/mimic_shmoof_CNNJoiLrgShmoofSmall.10K.csv.gz", compression="gzip")

Loading /Users/matsen/data/shmoof_pcp_2023-11-30_MASKED.csv.gz


In [4]:
data_nickname = "tangshm"
crepe_path = "../train/trained_models/cnn_joi_lrg-shmoof_small-fixed-0"
sim_df = sim_df_of_pcp_and_crepe(data_nickname, crepe_path, None)
sim_df.to_csv("~/data/mimic_tang_CNNJoiLrgShmoofSmall.10K.csv.gz", compression="gzip")

Loading /Users/matsen/data/tang-deepshm_size2_edges_22-May-2023.branch_length.csv.gz
